In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
import tensorflow as tf
import numpy as np
import skimage.io
import sklearn.decomposition
import matplotlib.pyplot as plt
import pathlib
import tqdm
import pickle
import annotation, misc, hyspec_cnn, hyspec_io, hyspec_ml, preprocess
import datetime

In [3]:
# Disable GPUs
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'
print(tf.version.VERSION)

2.12.0


In [4]:
# Paths
train_base = pathlib.Path('/media/mha114/Massimal/Vega_Sola/Hyperspectral/20220823/Area/3a_PCA_TrainValidationSplit/Training/')
val_base = pathlib.Path('/media/mha114/Massimal/Vega_Sola/Hyperspectral/20220823/Area/3a_PCA_TrainValidationSplit/Validation/')

train_im_path = train_base / 'PCA-Images'
train_tile_path = train_base / 'PCA-Tiles'
train_tile_path.mkdir(exist_ok=True)

val_im_path = val_base / 'PCA-Images'
val_tile_path = val_base / 'PCA-Tiles'
val_tile_path.mkdir(exist_ok=True)

In [5]:
def extract_pca_tiles(dataset_dir_path):
    TILE_SHAPE = (128,128)
    dataset_paths = [path for path in dataset_dir_path.glob('*')]
    tiles_dataset_list = []
    for dataset_path in tqdm.tqdm(dataset_paths):
        dataset = tf.data.Dataset.load(str(dataset_path))

        # Split into tiles
        pca_im, label_im = [numpy_data for numpy_data in dataset.as_numpy_iterator()][0]
        pca_tiles, label_tiles = hyspec_cnn.labeled_image_to_tensor_tiles(pca_im,label_im,TILE_SHAPE)

        # Create dataset from tiles
        tile_image_names = tf.convert_to_tensor([dataset_path.name for i in range(len(pca_tiles))])
        tiles_dataset_list.append(tf.data.Dataset.from_tensor_slices((pca_tiles,label_tiles,tile_image_names)))

    # Concatenate tiles
    tiles_concat = tiles_dataset_list[0]
    for tiles in tiles_dataset_list[1:]:
        tiles_concat = tiles_concat.concatenate(tiles)
    return tiles_concat

In [6]:
# Extract training tiles
train_tiles_dataset = extract_pca_tiles(train_im_path)
train_tiles_dataset.save(str(train_tile_path / '20220823_Vega_Sola_Train_Tiles'))

100%|███████████████████████████████████████████| 71/71 [00:32<00:00,  2.15it/s]


In [7]:
val_tiles_dataset = extract_pca_tiles(val_im_path)
val_tiles_dataset.save(str(val_tile_path / '20220823_Vega_Sola_Val_Tiles'))

100%|███████████████████████████████████████████| 10/10 [00:19<00:00,  1.99s/it]


In [8]:
# for pca_tile, label_tile, im_name in tiles_dataset.skip(100).take(5).as_numpy_iterator():
#     plt.imshow(pca_tile[:,:,0:3])
#     plt.show()
#     plt.imshow(label_tile)
#     plt.show()
#     print(im_name)